# Intro to Interactive Map Layers

**This notebook shows how to perform read & write operations on Interactive Map Layers.**

### Dependencies

* Datasets: Two example datasets will be stored into newly created layers.
* Language: Python

### Test data

Two free datasets (both GeoJSON) are written to interactive map layers.
One contains almost all
[world countries](https://github.com/johan/world.geo.json/blob/master/countries.geo.json)
as Polygon or MultiPolygon geometries, and the other contains the locations
of a number of
[volcanoes](https://earthworks.stanford.edu/download/file/harvard-glb-volc-geojson.json)
all over the world.

### Workflow

- Import packages, incl. HERE Python SDK ones
- Download external datasets
- Create catalog and layers
- Write GeoJSON data into layers
- Show statistical information of layers
- Get single feature from a layer
- Get multiple features from a layer
- Iterate over all the features in a layer
- Search features from a layer based on their properties
- Get features in a bounding box
- Spatial search by providing radius
- Spatial search using Geometry Intersection

## Import Packages

In [ ]:
from datetime import datetime
import json
import os

import requests
import shapely.geometry
from shapely.geometry import Point

from here.platform import Platform
from here.inspector import inspect, new_inspector
from here.inspector.styles import Color

## Download external datasets

In [ ]:
# Download volcanoe data from this source:
# Global GIS: volcanoes of the world; volcano basic data. [Shapefile].
# American Geological Institute. Retrieved from
# https://earthworks.stanford.edu/catalog/harvard-glb-volc 

with open("harvard-glb-volc-geojson.json", "w") as f:
    text = requests.get("https://earthworks.stanford.edu/download/file/harvard-glb-volc-geojson.json").text
    f.write(text)

In [ ]:
with open("countries.geo.json", "w") as f:
    text = requests.get("https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json").text
    f.write(text)

## Quick-inspect the sample datasets 

In [ ]:
countries_path = "countries.geo.json"
volcanoes_path = "harvard-glb-volc-geojson.json"

insp = new_inspector()
insp.add_features(json.load(open(countries_path)), name="Countries", style=Color.BLUE)
insp.add_features(json.load(open(volcanoes_path)), name="Volcanoes", style=Color.RED)
insp.show()

## Setup

In [ ]:
platform = Platform()

In [ ]:
# create a timestamp/hash-like ID for the catalog ID
ts = datetime.utcnow().isoformat()[:19]
ts = ts.replace("-", "").replace(":", "").replace("T", "").lower()
print(ts)

## Create resources on the platform

### Create catalog

In [ ]:
cat = platform.create_catalog(f"c{ts}", f"c{ts}", f"c{ts}", f"c{ts}")

### Add new Interactive Map layers to catalog

In [ ]:
countries_layer_details = {
    "id": "countries",
    "name": "countries",
    "summary": "Borders of world countries.",
    "description": "Borders of world countries.",
    "layerType": "interactivemap"
}

In [ ]:
volcanoes_layer_details = {
    "id": "volcanoes",
    "name": "volcanoes",
    "summary": "Volcanoes of the world",
    "description": "Volcanoes of the world",
    "layerType": "interactivemap"
}

In [ ]:
platform.update_catalog(
    hrn=cat.hrn, 
    id=f"c{ts}", 
    name="Countries and volcanoes", 
    summary="A catalog for countries and volcanoes.", 
    description="A catalog for countries and volcanoes.", 
    layers=[countries_layer_details, volcanoes_layer_details]
)

In [ ]:
# Global GIS: volcanoes of the world; volcano basic data. [Shapefile].
# American Geological Institute. Retrieved from
# https://earthworks.stanford.edu/catalog/harvard-glb-volc 

volcanoes_layer = cat.get_layer("volcanoes")
volcanoes_layer.write_features(from_file=volcanoes_path)

In [ ]:
countries_layer = cat.get_layer("countries")
countries_layer.write_features(from_file=countries_path)

## Show statistical information of layer

This shows some statitics for for the data in the newly created layers.

In [ ]:
volcanoes_layer.statistics

In [ ]:
countries_layer.statistics

## Get/inspect single feature from layer

In [ ]:
feature = countries_layer.get_feature(feature_id="IND")
inspect(feature, name="Selected feature")

## Get/inspect multiple features from layer

In [ ]:
features = countries_layer.get_features(feature_ids=["IND", "CHN"])
inspect(features, name="Selected features")

## Iterate over & inspect all features in a layer

In [ ]:
features = list(countries_layer.iter_features())
inspect(features, name="All the features")

## Search features from layer based on their properties

In [ ]:
searched_features = countries_layer.search_features(
    params={"p.name": ["India", "United States of America", "Australia", "Germany"]}
)
inspect(searched_features, name="Searched features")

## Get features in a bounding box

In [ ]:
insp = new_inspector()

bbox = (68.1766451354, 7.96553477623, 97.4025614766, 35.4940095078)
bbox_features = countries_layer.get_features_in_bounding_box(bbox)
insp.add_features(bbox_features, name="Features in bounding box")

box_polygon = shapely.geometry.box(*bbox)
insp.add_features(box_polygon, name="Search box", style=Color.GRAY)

insp.show()

## Spatial search within radius

In [ ]:
insp = new_inspector()

radius_search = countries_layer.spatial_search(lat=0, lng=30, radius=1000000)
insp.add_features(radius_search, name="Features within radius")

circle = Point(30, 0).buffer(10)
insp.add_features(circle, name="Search radius", style=Color.GRAY)

insp.show()

## Spatial search using Geometry Intersection

In [ ]:
feature = countries_layer.get_feature(feature_id="USA")
geometry = feature["geometry"]
geometry_search = volcanoes_layer.spatial_search_geometry(geometry=geometry)
inspect(geometry_search, name="Volcanoes in USA", style=Color.RED)

## Clean up

Here, if we like, we can remove all resources generated by this notebook (both locally and on the platform).

In [ ]:
platform.delete_catalog(cat.hrn)
os.remove(countries_path)
os.remove(volcanoes_path)

<span style="float:left; margin-top:3px;"><img src="https://www.here.com/themes/custom/here_base_theme_v2/logo.svg" alt="HERE Logo" height="60" width="60"></span><span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2021 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential information which may not be disclosed to others without the prior written consent of HERE.</sub></span>